### K-means (Step.2) クラスタリング, 初期ラベルを変えたときの挙動

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

def dic_color(img):
    img//= 64
    img = img*64+32
    return img

def get_DB():
    # get image path
    train = glob('../dataset/train_*')
    train.sort()
    
    # prepare database
    db = np.zeros((len(train), 13), dtype=np.int32)
    
    # prepare path database
    # 何のために使っているか考えよう
    pdb = []
    
    # each train
    for i, path in enumerate(train):
        # image read
        img = dic_color(cv2.imread(path))
        # histogram
        for j in range(4):
            db[i,j] = len(np.where(img[...,0]==(j*64+32))[0])
            db[i,j+4] = len(np.where(img[...,1]==(j*64+32))[0])
            db[i,j+8] = len(np.where(img[...,2]==(j*64+32))[0])
            
        # get class label
        if 'akahara' in path:
            cls = 0
        elif 'madara' in path:
            cls = 1
            
        # store class label
        db[i, -1] = cls
        
        # add image path
        pdb.append(path)
        
    return db, pdb

def k_means_step2(db, pdb, Class=2):
    # copy database
    feats = db.copy()  # (10, 13)

    # initiate random seed
     # 初期値によって挙動がどういうふうに代わるか確認してみる　正しく分類されることはあるのだろうか
    np.random.seed(1)
    
    # assign random class
    for i in range(len(feats)):  # train画像の個数分の繰り返し
        if np.random.random() < 0.41:
            feats[i, -1] = 0
        else:
            feats[i, -1] = 1
     
    while True:
        # prepare gravity
        # 各色ごとの重心を求めるの？
        gs = np.zeros((Class, 12), dtype=np.float32)  # (2,12)
        is_changed = 0
    
        # get gravity
        for i in range(Class):
            print(np.where(feats[:,-1]==i))
            #print(np.where(feats[:,-1]==i).shape)
            gs[i] = np.mean(feats[np.where(feats[:,-1]==i)[0], :12], axis=0)
        
        # re-labering  各サンプルについて
        for i in range(len(feats)):
            # get distance each nearest gravity
            dis = np.sqrt(np.sum(np.square(np.abs(gs-feats[i,:12])), axis=1))
            
            # get new lable
            pred = np.argmin(dis, axis=0)  # 0, 1  最小のインデックスがそのままラベルになる
            
            # if label is different from old label
            if int(feats[i,-1])!=pred:
                is_changed = 1
                feats[i,-1] = pred
                
        if is_changed < 1:
            break
            
            
    for i, path in enumerate(pdb):
        print(path, 'Pred : ', feats[i,-1])
    
    
    
db, pdb = get_DB()


k_means_step2(db, pdb, Class=2)

              
        

(array([2, 3, 4, 5, 6, 7, 8]),)
(array([0, 1, 9]),)
(array([2, 4, 5, 6, 7, 8, 9]),)
(array([0, 1, 3]),)
(array([4, 5, 6, 7, 8, 9]),)
(array([0, 1, 2, 3]),)
(array([5, 6, 7, 8, 9]),)
(array([0, 1, 2, 3, 4]),)
../dataset/train_akahara_1.jpg Pred :  1
../dataset/train_akahara_2.jpg Pred :  1
../dataset/train_akahara_3.jpg Pred :  1
../dataset/train_akahara_4.jpg Pred :  1
../dataset/train_akahara_5.jpg Pred :  1
../dataset/train_madara_1.jpg Pred :  0
../dataset/train_madara_2.jpg Pred :  0
../dataset/train_madara_3.jpg Pred :  0
../dataset/train_madara_4.jpg Pred :  0
../dataset/train_madara_5.jpg Pred :  0
